## RUHANI FAIHEEM RAHMAN

## Assignment 04


In [1]:
import nltk
import re
import math

Reading file

In [2]:
def read_file(file_name='tweets_corpus.txt'):
    with open(file_name,'r',encoding='utf-8') as file:
        text = file.read()
    text = text.translate(str.maketrans('', '', '!"#$%&\()*+,.:;<=>?@[\\]^_`{|}~')).lower()
    return text.split('\n')

Creating the dataset for processing. Used Dictionary data structures.

In [35]:
def create_dataset(file_name='tweets_corpus.txt'):
    text = read_file(file_name)
    dataset = dict()
    unique_words = ''
    for i in text:
        data = i.split('\t',1)
        if data.__len__() == 2:
            dataset[data[0]] = data[1]
            unique_words += data[1] + ' '
    unique_words = {words:list() for words in nltk.FreqDist(unique_words.split()).keys()}
    return dataset, unique_words

Code for printing dictionary

In [19]:
def print_dict(dict):
    for key, val in dict.items():
        print(key,':',val)

Method for creating inverted matrix

In [20]:
def create_inverted_index(dataset, unique_words):
    for key, val in dataset.items():
        for words in val.split():
            inverted_list = unique_words[words]
            if type(inverted_list) == 'NonType':
                unique_words[words] = [key]
            else:
                inverted_list.append(key)
                unique_words[words] = list(set(inverted_list))
    print_dict(unique_words)
    return unique_words

Merge Algorithm. Folllwed the optimzied query algorithm

In [21]:
def merge_query(query_str, inverted_list, dataset):
    universal_doc_list = [key for key in dataset.keys()]
    queries = re.findall('\((.*?)\)', query_str)
    if len(queries) != 0:
        temp_str = query_str
        for qq in queries:
            temp_str = temp_str.replace(qq,'***')
        str_tokenize = temp_str.split()
        result_list = [i for i in range(0, len(str_tokenize))]

        count = 0
        for tokens in enumerate(str_tokenize):
            if tokens[1] == '(***)':
                temp_str_tokenize = queries[count].split()
                temp_result_list = [i for i in range(0, len(temp_str_tokenize))]
                result_list[tokens[0]] = process_string(temp_str_tokenize, temp_result_list, inverted_list, dataset)
                count+=1
        return process_string(str_tokenize, result_list, inverted_list, dataset)
    else:
        str_tokenize = query_str.split()
        result_list = [i for i in range(0, len(str_tokenize))]
        return process_string(str_tokenize, result_list, inverted_list, dataset)

Finding query, get min query length, process lists for query

In [22]:
def process_string(str_tokenize, result_list, inverted_list, dataset):
    i = 0
    while(i < len(str_tokenize)):
        if str_tokenize[i] == 'NOT':
            result_list[i+1] = NOT(str_tokenize[i+1], inverted_list, dataset)
            str_tokenize.remove('NOT')
            result_list.remove(i)
            i-=1
        if str_tokenize[i] not in ['AND','OR'] and str_tokenize[i] != '(***)':
            result_list[i] = inverted_list[str_tokenize[i]]
        i+=1

    while(len(str_tokenize) > 1):
        min_index,str_tokenize,result_list = find_min_index(str_tokenize,result_list)
        str_tokenize, result_list = proceed_Operation(min_index,str_tokenize,result_list)
    return result_list[0]

Finds minimum index

In [37]:
def find_min_index(str_tokenize, result_list):
    min_val = 100000
    min_index = -1
    for i in range(0, len(result_list)):
        if str_tokenize[i] == 'AND':
            if min_val >= min(len(result_list[i-1]),len(result_list[i-1])):
                min_val = min(len(result_list[i-1]),len(result_list[i-1]))
                min_index = i
                result_list[i] = min(len(result_list[i-1]),len(result_list[i-1]))
        elif str_tokenize[i] == 'OR':
            if min_val >= max(len(result_list[i-1]),len(result_list[i-1])):
                min_val = max(len(result_list[i-1]),len(result_list[i-1]))
                min_index = i
                result_list[i] = max(len(result_list[i-1]),len(result_list[i-1]))
    return min_index, str_tokenize, result_list

Process AND OR operation

In [24]:
def proceed_Operation(index, str_tokenize, result_list):
    if str_tokenize[index] == 'AND':
        result_list[index-1] = AND(result_list[index-1], result_list[index+1])
        str_tokenize[index-1] = str_tokenize[index-1] + ' ' + str_tokenize[index] + ' ' + str_tokenize[index+1]
    elif str_tokenize[index] == 'OR':
        result_list[index-1] = OR(result_list[index-1], result_list[index+1])
        str_tokenize[index-1] = str_tokenize[index-1] + ' ' + str_tokenize[index] + ' ' + str_tokenize[index+1]

    return str_tokenize[:index] + str_tokenize[index+2:], result_list[:index] + result_list[index+2:]

Method for calculating OR AND NOT

In [44]:
def OR(list1, list2):
    return list(set(list1).union(set(list2)))

def AND(list1, list2):
    return list(set(list1).intersection(set(list2)))

def NOT(word, inverted_list, dataset):
    lists = inverted_list[word]
#     total_doc_list = [key for key in dataset.keys() if key not in lists]
    total_doc_list = [key for key in dataset.keys()]
    for a in lists:
        total_doc_list.remove(a)
    return total_doc_list

Calculating TF, IDF and TFIDF

In [26]:
def TF(word, doc_ID, dataset):
    return dataset[doc_ID].split().count(word) / len(dataset[doc_ID].split())

def IDF(word, dataset, inverted_index):
    return math.log10(len(dataset)/len(inverted_index[word]))

def TF_IDF(word, doc_ID, dataset, inverted_index):
    return TF(word, doc_ID, dataset) * IDF(word, dataset, inverted_index) * 1.0

TF_IDF Query Algorithm

In [27]:
def TF_IDF_QUERY(query_str, inverted_list, dataset):
    query_result = merge_query(query_str, inverted_list, dataset)
    query_str = query_str.translate(str.maketrans('', '', '!"#$%&\()*+,.:;<=>?@[\\]^_`{|}~')).lower()
    query_list = [words for words in query_str.split() if words not in ['AND', 'OR', 'NOT']]

    TF_IDF_Dict = dict()
    for doc_ID in query_result:
        score = 0.0
        for words in query_list:
            score += TF_IDF(words, doc_ID, dataset, inverted_list)
        TF_IDF_Dict[doc_ID] = score
    for key, val in TF_IDF_Dict.items():
        print(key, ':', val, ':', dataset[key])
    return TF_IDF_Dict

In [28]:
def print_result(lists, dataset):
    for ll in lists:
        print(ll, ':', dataset[ll])

### Problem 01

In [39]:
dataset, unique_words = create_dataset()
inverted_list = create_inverted_index(dataset, unique_words)

bandaging : ['81499877556760576']
up : ['81644157432643584', '81499877556760576']
my : ['81644157432643584', '81499877556760576', '81656304107651072', '81507775422791680', '81673244926685184']
paper-cuts : ['81499877556760576']
having : ['81499877556760576']
cheesecake : ['81499877556760576', '81600113016971264', '81716618236928000']
for : ['81499877556760576', '81715158593966080', '81716618236928000']
dinner : ['81499877556760576']
and : ['81587643376336896', '81716618236928000', '81499877556760576', '81715158593966080', '81656304107651072', '81736742478155778', '81673244926685184']
calling : ['81499877556760576']
it : ['81499877556760576', '81600113016971264', '82650970722533376', '81715158593966080', '81842384404623360']
a : ['81587643376336896', '81623945064890368', '81644157432643584', '81582996083326976', '81499877556760576', '81715158593966080', '81534165975171072', '81535634019323904', '81600113016971264', '81673244926685184']
night : ['81499877556760576', '82650970722533376']


Example selected for testing my code.

In [30]:
query = [
    'egg OR cheese',
    'egg AND NOT cheese',
    'chocolate AND strawberry',
    'eggs AND cheese AND bacon AND chocolate',
    'chocolate OR strawberry OR eggs OR NOT bacon',
    'eggs AND cheese OR strawberry AND NOT chocolate',
    '(eggs AND cheese) OR (eggs AND bacon)'
]

### Problem 02

In [43]:
for q in query:
    print('Query: ',q)
    print('Results:')
    results = merge_query(query[6], inverted_list,dataset)
    print_result(results, dataset)
    print('\n\n')

Query:  egg OR cheese
Results:
85032815321825280 : cheese hashbrowns  turkey bacon  veggie tofu scramble  rice  french toast  scrambled eggs  strawberries  cantaloupe  tagfinalhashtags
86441828815089664 : rt tagusername  rt tagusername  pancakes  bacon  eggs w/ cheese   hashbrown casserole on deck taghashtags  i want sum  ok it's good too
81736742478155778 : made myself some scrambled eggs with cheese and bacon bits
81673244926685184 : i think i want some cheese eggs and pancakes  but will i cook  where's my gf  this ain't right to make such a hard decision



Query:  egg AND NOT cheese
Results:
85032815321825280 : cheese hashbrowns  turkey bacon  veggie tofu scramble  rice  french toast  scrambled eggs  strawberries  cantaloupe  tagfinalhashtags
86441828815089664 : rt tagusername  rt tagusername  pancakes  bacon  eggs w/ cheese   hashbrown casserole on deck taghashtags  i want sum  ok it's good too
81736742478155778 : made myself some scrambled eggs with cheese and bacon bits
81673244

In [41]:
for q in query:
    print('Query: ',q)
    print('Results:')
    TF_IDF_QUERY(q, inverted_list,dataset)    
    print('\n\n') 

Query:  egg OR cheese
Results:
81587643376336896 : 0.09744648969972605 : rt tagusername  i want a steak and cheese egg roll right now 
86441828815089664 : 0.012679423304989012 : rt tagusername  rt tagusername  pancakes  bacon  eggs w/ cheese   hashbrown casserole on deck taghashtags  i want sum  ok it's good too
81582996083326976 : 0.01065071557619077 : tagusername 1st off i'm like 1 year younger than u  2nd age is just a number  3rd ima cater ur wedding wit patty n cheese
81503002321616896 : 0.08995060587667021 : bacon/cheddar slider topped w/fried egg  blue cheese slider topped w/avocado  purple cherokee tomato
81577509950459904 : 0.13313394470238463 : mmmm cheese
85032815321825280 : 0.017751192626984616 : cheese hashbrowns  turkey bacon  veggie tofu scramble  rice  french toast  scrambled eggs  strawberries  cantaloupe  tagfinalhashtags
82650970722533376 : 0.05568370839984346 : i went swimming  then ate asparagus bacon egg cheese biscuit goodness  then watched date night  it was  it